## Codon and Motif Shuffle Reproducibility Workflow
This notebook documents the process of generating new sequence datasets for testing SpliceAI.

Objective
To investigate how SpliceAI relies on different sequence features (e.g., nucleotide structure, codon structure), we:

Shuffle nucleotide sequences in different ways
Preserve structures like splice junctions
Generate perturbed versions of the original transcriptome
Workflow Overview
1. Load Canonical Transcript Dataset
Read exon/intron boundaries from canonical .bed files
Extract corresponding sequences from a reference FASTA
2. Apply Shuffling Strategies
Nucleotide Shuffle: Randomize exon sequences fully (baseline destruction)
Codon Shuffle: Shuffle only within codon triplets, preserving reading frame
3. Reconstruct FASTA Files
Replace original sequences with shuffled versions
Ensure intron/exon structure and transcript orientation is preserved
4. Write New Datasets
Save to FASTA, BED, or HDF5 files for SpliceAI input
Include aligned inputs for comparative testing
Outputs
Shuffled FASTA files for input to SpliceAI as the 'splice table' argument
This notebook serves as documentation for dataset perturbation and reproducibility of the shuffling experiment.

In [ ]:
import pysam
import pandas as pd


reference_genome_path = "/mnt/lareaulab/sdahiyat/hg19.fa" # my reference genome
canonical_dataset_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_dataset_created.txt"
output_file_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_sequence_unflanked_unshuffled.txt"

# Open the reference genome
reference_genome = pysam.FastaFile(reference_genome_path)

# Load canonical dataset
canonical_df = pd.read_csv(
    canonical_dataset_path,
    sep="\t",
    header=None,
    names=["geneName", "chrom", "strand", "chromStart", "chromEnd", "exonEnds", "exonStarts"]
)

# Create the canonical sequence file **without flanked 'N's**
with open(output_file_path, "w") as output_file:
    for _, row in canonical_df.iterrows():
        chrom = row["chrom"]
        chrom_start = row["chromStart"]
        chrom_end = row["chromEnd"]
        strand = row["strand"]

        # Fetch the **exact** sequence without adding flanks
        core_sequence = reference_genome.fetch(chrom, chrom_start-1, chrom_end)  # 0-based index

        # Reverse complement if on the negative strand
        if strand == "-":
            core_sequence = core_sequence.translate(str.maketrans("ATCG", "TAGC"))[::-1]

        # Write to output file with canonical transcript ID
        output_file.write(f"{chrom}:{chrom_start}-{chrom_end}\t{core_sequence}\n")

print(f" Canonical sequence file created at {output_file_path} (without flanking 'N's)")


## calculate nucleotide frequencies of intron sequences

In [ ]:
from collections import Counter

valid_nucleotides = {"A", "C", "G", "T"} 
input_file = "extracted_intron_sequences.txt" # separate file of just intron sequences
nucleotide_counts = Counter()

with open(input_file, "r") as file:
    for line in file:
        if not line.startswith(">"): 
            sequence = [char for char in line.strip().upper() if char in valid_nucleotides]
            nucleotide_counts.update(sequence)

total_bases = sum(count for base, count in nucleotide_counts.items() if base != "N")

frequencies = {base: count / total_bases for base, count in nucleotide_counts.items() if base != "N"}
print("Nucleotide Frequencies (excluding N):")
for base, freq in sorted(frequencies.items()):
    print(f"{base}: {freq:.4f}")


In [ ]:
import random

# Define nucleotide frequencies
nucleotide_frequencies = {
    "A": 0.2781,
    "C": 0.2025,
    "G": 0.2120,
    "T": 0.3075
}
nucleotides = list(nucleotide_frequencies.keys())
weights = list(nucleotide_frequencies.values())

def generate_random_sequence(length):
    return ''.join(random.choices(nucleotides, weights=weights, k=length))

def shuffle_sequences(canonical_sequences_file, output_file, junctions):
    with open(canonical_sequences_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            line = line.strip()
            if not line:
                continue

            try:
                header, sequence = line.split("\t")
            except ValueError:
                print(f"Skipping malformed line: {line}")
                continue

            transcript_key = header

            if transcript_key in junctions:
                seq_start = int(transcript_key.split(":")[1].split("-")[0])
                adjusted_junctions = [
                    (start - seq_start, end - seq_start)
                    for start, end in junctions[transcript_key]
                ]

                exons_introns = []
                last_pos = 0
                for exon_start, exon_end in adjusted_junctions:
                    exons_introns.append(sequence[last_pos:exon_start])

                    # Shuffle exon
                    exon_length = exon_end - exon_start
                    fake_exon = generate_random_sequence(exon_length)
                    exons_introns.append(fake_exon)

                    last_pos = exon_end

                exons_introns.append(sequence[last_pos:])  
                shuffled_sequence = ''.join(exons_introns)
            else:
                shuffled_sequence = generate_random_sequence(len(sequence))

            outfile.write(f"{header}\t{shuffled_sequence}\n")



canonical_dataset_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_dataset_created.txt"
canonical_sequences_file = "/mnt/lareaulab/sdahiyat/illumina/canonical_sequence_unflanked_unshuffled.txt"
output_file_path = "/mnt/lareaulab/sdahiyat/illumina/shuffled_nt_new.txt"

junctions = parse_junctions(canonical_dataset_path)

shuffle_sequences(canonical_sequences_file, output_file_path, junctions)

print(f"Shuffled sequences written to {output_file_path}.")


In [ ]:
input_file = "/mnt/lareaulab/sdahiyat/illumina/shuffled_nt_new.txt"
output_file = "/mnt/lareaulab/sdahiyat/illumina/flanked_shuffled_nt.txt"

# Define the flanking sequence (5000 Ns)
flank_length = 5000
flanking_seq = "N" * flank_length

# Process the input file and write the modified sequences to output
with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        
        try:
            header, sequence = line.split("\t")
        except ValueError:
            print(f"Skipping malformed line: {line}")
            continue
        
        # Extract chromosome, start, and end positions
        if ":" in header and "-" in header:
            chrom, positions = header.split(":")
            start, end = map(int, positions.split("-"))
            
            # Adjust start and end positions
            new_start = start - 5001 
            new_end = end + 5000
            
            # Construct new header
            new_header = f"{chrom}:{new_start}-{new_end}"
            
            # Construct new sequence with flanking Ns
            flanked_sequence = flanking_seq + sequence + flanking_seq
            
            # Write to output file
            outfile.write(f"{new_header}\t{flanked_sequence}\n")
        else:
            print(f"Skipping improperly formatted header: {header}")

print(f"Flanked sequences written to {output_file}.")


## Codon Shuffling Pipeline Below

In [ ]:
import random
from collections import Counter
def parse_junctions(file_path):
    junctions = {}
    with open(file_path, "r") as f:
        for line_number, line in enumerate(f, start=1):
            fields = line.strip().split("\t")
            if len(fields) < 8:
                continue

            chrom = fields[2]
            try:
                start = int(fields[4])
                end = int(fields[5])
            except ValueError:
                continue

            try:
                exon_starts = list(map(int, fields[6].split(",")))
                exon_ends = list(map(int, fields[7].split(",")))
            except ValueError:
                continue

            if len(exon_starts) != len(exon_ends):
                continue

            transcript_key = f"{chrom}:{start}-{end}"
            junctions[transcript_key] = list(zip(exon_starts, exon_ends))
    return junctions

def load_sequences(file_path):
    sequences = {}
    with open(file_path, "r") as infile:
        for line in infile:
            line = line.strip()
            if not line:
                continue
            try:
                header, sequence = line.split("\t")
                sequences[header] = sequence
            except ValueError:
                continue
    return sequences

# compute global codon frequencies from all exons
def compute_global_codon_frequencies(sequences, junctions):
    codon_list = []
    for transcript_key in sequences:
        if transcript_key not in junctions:
            continue
        seq = sequences[transcript_key]
        seq_start = int(transcript_key.split(":")[1].split("-")[0])
        for start, end in junctions[transcript_key]:
            start_rel = start - seq_start
            end_rel = end - seq_start
            exon = seq[start_rel:end_rel].upper()
            codon_list.extend([
                exon[i:i+3] for i in range(0, len(exon) - 2, 3)
                if len(exon[i:i+3]) == 3 and all(c in "ACGT" for c in exon[i:i+3])
            ])
    codon_counts = Counter(codon_list)
    total = sum(codon_counts.values())
    return list(codon_counts.keys()), [codon_counts[c] / total for c in codon_counts]

# generate shuffled exon by sampling codons
def generate_shuffled_exon(length, codons, weights):
    if length <= 0:
        return ""
    num_codons = length // 3
    shuffled = ''.join(random.choices(codons, weights=weights, k=num_codons))
    remainder = length % 3
    if remainder > 0:
        shuffled += ''.join(random.choices("ACGT", k=remainder))
    return shuffled

# shuffle while preserving introns 
def shuffle_sequences(canonical_sequences_file, output_file, junctions, codons, weights):
    with open(canonical_sequences_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            line = line.strip()
            if not line:
                continue

            try:
                header, sequence = line.split("\t")
            except ValueError:
                continue

            transcript_key = header

            if transcript_key in junctions:
                seq_start = int(transcript_key.split(":")[1].split("-")[0])
                adjusted_junctions = [
                    (start - seq_start, end - seq_start)
                    for start, end in junctions[transcript_key]
                ]

                exons_introns = []
                last_pos = 0
                for exon_start, exon_end in adjusted_junctions:
                    exons_introns.append(sequence[last_pos:exon_start])
                    exon_len = exon_end - exon_start
                    shuffled_exon = generate_shuffled_exon(exon_len, codons, weights)
                    exons_introns.append(shuffled_exon)
                    last_pos = exon_end

                exons_introns.append(sequence[last_pos:])
                shuffled_sequence = ''.join(exons_introns)
            else:
                shuffled_sequence = generate_shuffled_exon(len(sequence), codons, weights)

            outfile.write(f"{transcript_key}\t{shuffled_sequence}\n")

canonical_dataset_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_dataset_created.txt"
canonical_sequences_file = "/mnt/lareaulab/sdahiyat/illumina/canonical_sequence_unflanked_unshuffled.txt"
output_file_path = "/mnt/lareaulab/sdahiyat/illumina/shuffled_codons_from_codon_freq.txt"

#main pipeline 
junctions = parse_junctions(canonical_dataset_path)
sequences = load_sequences(canonical_sequences_file)
codons, weights = compute_global_codon_frequencies(sequences, junctions)

print(f"Parsed {len(junctions)} junctions and {len(sequences)} sequences")
print(f"Shuffling using {len(codons)} codons")
shuffle_sequences(canonical_sequences_file, output_file_path, junctions, codons, weights)
print(f"Shuffled sequences written to {output_file_path}")


In [ ]:
input_file = "/mnt/lareaulab/sdahiyat/illumina/shuffled_codons_from_codon_freq.txt"
output_file = "/mnt/lareaulab/sdahiyat/illumina/flanked_shuffled_codons_423version.txt"

# define the flanking sequence of 5000 Ns
flank_length = 5000
flanking_seq = "N" * flank_length

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        line = line.strip()
        if not line:
            continue  
        
        try:
            header, sequence = line.split("\t")
        except ValueError:
            print(f"Skipping malformed line: {line}")
            continue
        
        # Extract chromosome, start, and end positions
        if ":" in header and "-" in header:
            chrom, positions = header.split(":")
            start, end = map(int, positions.split("-"))
            
            # Adjust start and end positions
            new_start = start - 5001 
            new_end = end + 5000
            
            new_header = f"{chrom}:{new_start}-{new_end}"
            
            flanked_sequence = flanking_seq + sequence + flanking_seq
            
            outfile.write(f"{new_header}\t{flanked_sequence}\n")
        else:
            print(f"Skipping improperly formatted header: {header}")

print(f"Flanked sequences written to {output_file}.")
